# STAC EOPF Product

Invoke a Common Workflow Language CommandLineTool to generate the EOPF Product described using STAC

This notebook is linked to: https://eoap.github.io/zarr-cloud-native-format/cwl-cli/stac-eopf-product



## Setup

In [11]:
export WORKSPACE=/workspace/zarr-cloud-native-format
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}

curl -q -L https://github.com/eoap/zarr-cloud-native-format/releases/download/0.5.0/app-water-bodies.0.5.0.cwl > ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl 2> /dev/null

## Run the STAC EOPF Product generation CommandLineTool

Inspect and use `cwltool` to run the STAC EOPF Product generation definition:


In [13]:
cat ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl | yq e '.["$graph"][8]' -

class: CommandLineTool
id: stac-eopf-product
label: Create a EOPF Zarr store from a STAC catalog
doc: Create a EOPF Zarr store from a STAC catalog with COG products
requirements:
  InlineJavascriptRequirement: {}
  EnvVarRequirement:
    envDef:
      PATH: /usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
      PYTHONPATH: /app
  ResourceRequirement:
    coresMax: 1
    ramMax: 512
hints:
  DockerRequirement:
    dockerPull: docker.io/library/zarr-eopf-product
baseCommand: ["python", "-m", "app"]
arguments: []
inputs:
  stac_catalog:
    type: Directory
    inputBinding:
      prefix: --stac-catalog
  collection_id:
    type: string
    inputBinding:
      prefix: --collection-id
    default: "water-bodies-detection"
outputs:
  eopf_product_stac_catalog:
    outputBinding:
      glob: .
    type: Directory


Run the CWL description, but first prepare the parameters.

The previous step generated the water bodies detection geotif:

In [ ]:
cat stac-generation-results.json 

Let's build the job parameters file

In [ ]:
cat <<EOF > stac-zarr-generation-params.yaml
stac_catalog: 
  class: Directory
  path: $( cat stac-generation-results.json | jq -r '.temp_stac_catalog.path' )
EOF

cat stac-zarr-generation-params.yaml | yq .

In [ ]:
cwltool \
    --podman \
    --outdir ${WORKSPACE}/runs \
    ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl#stac-eopf-product \
    stac-zarr-generation-params.yaml > stac-eopf-product-generation-results.json 2> stac-eopf-product-generation.log

Let's look at the content of the stderr:

In [ ]:
cat stac-eopf-product-generation.log | egrep -v "WARNING|JSHINT"

Let's inspect the stdout produced. 

In [ ]:
cat stac-eopf-product-generation-results.json | jq . -